<a href="https://colab.research.google.com/github/EncodingDavid/tensorflow/blob/master/TensorFlow%E5%AE%9E%E7%8E%B0%E8%87%AA%E7%BC%96%E7%A0%81%E5%99%A8%E5%8F%8A%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深度学习简介
深度学习在早期一度被认为是一种无监督的特征学习，模仿人脑对特征逐层抽象提取的过程。有亮点很重要。一是无监督学习。即我们不需要标注数据就可以对数据进行一定程度的学习，这种学习是对数据内容的组织形式的学习，提取的是频繁出现的特征；而是逐层抽象，特征是需要不断抽象的。

自编码器：可以使用自身的高阶特征编码自己，它的输入输出是一致的，即y=x，还有就是使用高阶重构自己，而不是复制像素点。

先用自编码器的方法进行无监督的预训练，提取特征并初始化权重，然后使用标注信息进行监督式训练

## 使用TensorFlow实现自编码器

In [0]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
def xavier_init(fan_in,fan_out,constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in,fan_out),minval = low,maxval = high,dtype = tf.float32)

In [0]:
# 定义一个去噪自编码class
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self,n_input,n_hidden,transfer_function = tf.nn.softplus,optimizer = tf.train.AdamOptimizer(),scale =0.1):
        self.n_input = n_input 
        self.n_hidden = n_hidden #只使用了一个隐含层
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32) #将scale参数做成一个placeholder
        self.training_scale = scale
        network_weights = self._initialize_weights() #接下来会定义_initialize_weights()函数
        self.weights = network_weights
        
        # 定义网络结构
        
    # 为x创建一个维度为n_input的placeholder。
    
    # 然后建立一个能提取特征的隐含层，我们先将输入x加上噪声，即`self.x+scale*tf.random_normal((n_input,))，
    
    # 然后用tf.matmul将加了噪声的输入与隐含层的权重w1相乘，
    
    # 并使用`tf.add`加上隐含层的偏置b1，最后使用`self.transfer`对结果进行激活函数处理。
    
    # 经过隐含层后，我们需要在输出层进行数据复原、重建操作。
    
        self.x = tf.placeholder(tf.float32,[None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + scale * tf.random_normal((n_input,)),self.weights['w1']),self.weights['b1'])) #建立一个能够提取特征的隐含层
        self.reconstruction = tf.add(tf.matmul(self.hidden,self.weights['w2']),self.weights['b2']) #在输出层进行数据复原，重建操作，即建立reconstruction层
       
       # 定义自编码器的损失函数，这里直接使用平方误差作为cost
        
    # 即用tf.substract计算输出与输入之差
    
    # 再使用tf.pow求差的平方，最后使用tf.reduce_sum求和即可得到平方误差。
    
    # 再定义训练操作为优化器self.optimizer对损失self.cost进行优化。
    
    # 最后创建Session，并初始化自编码器的全部模型参数
      
      
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0))
        self.optimizer = optimizer.minimize(self.cost)  

        init = tf.global_variables_initializer() #初始化自编码器的全部模型参数
        self.sess = tf.Session()
        self.sess.run(init)
        
    def _initialize_weights(self):
        all_weights = dict() #创建一个字典，存入w1,b1,w2,b2
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input,self.n_hidden)) #使用xavier_init函数初始化，返回一个比较适合于softplus等激活函数的权重初始分布
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input],dtype = tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype = tf.float32))
        return all_weights
      
      # 定义计算损失cost及执行一步训练的函数partial_fit。函数里只需让Session执行两个计算图的节点，分别是损失cost和训练过程optimizer
      
    def partial_fit(self,X):
        cost,opt = self.sess.run((self.cost,self.optimizer),feed_dict = {self.x:X,self.scale:self.training_scale})
        return cost
      # 我们也需要一个只求损失cost的函数calc_total_cost，这里就只让Session执行一个计算图节点self.cost
    def calc_total_cost(self,X):
        return self.sess.run(self.cost,feed_dict = {self.x:X,self.scale:self.training_scale})
      # 定义transform函数，它返回自编码器隐含层的输出结果。自编码器的隐含层的最主要的功能就是学习出数据中的高阶特征
    def transform(self,X):
      return self.sess.run(self.hidden)
    
      

定义reconstruct函数，它整体运行复原一遍，包括提取高阶特征和通过高阶特征复原数据，即包括transform和generate两块。输入数据是原数据，输出数据是复原后的数据

In [0]:
    def reconstruct(self,X):
        return self.sess.run(self.reconstruction,feed_dict = {self.x:X,self.scale:self.training_scale})

这里的getWeights函数是为了获取隐含层的权重w1

In [0]:
    def getWeights(self):
        return self.sess.run(self.weights['w1']) #作用是获取隐含层的权重w1

获取偏置函数

In [0]:
    def getBiases(self):
        return self.sess.run(sefl.weights['b1']) #作用是获取隐含层的偏执b1

至此，去噪自编码器的class就定义完了，包括神经网络的设计，权重的初始化，以及常用成员函数等

接下来依然使用TensorFlow提供的读取示例数据的函数载入MNIST数据集

In [47]:
mnist = input_data.read_data_sets('MNIST_data',one_hot = True)
#载入MNIST数据集

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


定义一个对训练、测试数据进行标准化的函数

In [0]:
def standard_scale(X_train,X_test):
    preprocessor = prep.StandardScaler().fit(X_train) #StandarScaler是sklearn.preprossing工具包里面的类，先在训练集上fit
    X_train = preprocessor.transform(X_train) #transform是返回隐含层的输出结果，所以X_train就完成了标准化处理
    X_test = preprocessor.transform(X_test)
    return X_train,X_test

再定义一个获取随机block数据的函数

In [0]:
def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data) - batch_size)#从0到len(data) - batch_size之间取一个随机数
    return data[start_index:(start_index + batch_size)]#从随机数的位置开始，顺序取一个 batch size的数据。这里是不放回的抽样

标准化变换

In [0]:
X_train,X_test = standard_scale(mnist.train.images, mnist.test.images)

定义几个常用参数，总训练样本数，最大训练次数为20，batch为128，并设置每隔一次就显示一次损失cost

In [0]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

创建一个AGN自编码器，定义模型输入节点数为784，自编码器的隐含层节点数为200，隐含层的激活函数为softplus，优化器optimizer为Adam且学习速率为0.001，同时将噪声的系数scale设为0.01

In [0]:
autoencoder  = AdditiveGaussianNoiseAutoencoder(n_input = 784,n_hidden = 200,transfer_function = tf.nn.softplus,optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),scale = 0.01)

# 开始训练！
- 在每一轮循环开始时，我们将平均损失avg_cost设为0，并计算总共需要的batch数（通过样本总数除以batch大小），注意这里使用的是不放回抽样，所以不能保证每一个样本都能参与训练
- 在每一个batch的循环中，先使用get_random_block_from_data函数随机抽取一个block的数据
- 然后使用成员函数partial_fit训练这个batch的数据并计算当前的cost，最后将当前的cost整合到avg_cost中


In [53]:
for epoch in range(training_epochs):
  avg_cost = 0.
  total_batch = int(n_samples / batch_size)
  for i in range(total_batch):
    batch_xs = get_random_block_from_data(X_train,batch_size)
    
    cost = autoencoder.partial_fit(batch_xs)
    avg_cost += cost / n_samples * batch_size
  if epoch % display_step == 0:
    print("Epoch:",'%04d' % (epoch + 1),"cost=","{:.9f}")

Epoch: 0001 cost= {:.9f}
Epoch: 0002 cost= {:.9f}
Epoch: 0003 cost= {:.9f}
Epoch: 0004 cost= {:.9f}
Epoch: 0005 cost= {:.9f}
Epoch: 0006 cost= {:.9f}
Epoch: 0007 cost= {:.9f}
Epoch: 0008 cost= {:.9f}
Epoch: 0009 cost= {:.9f}
Epoch: 0010 cost= {:.9f}
Epoch: 0011 cost= {:.9f}
Epoch: 0012 cost= {:.9f}
Epoch: 0013 cost= {:.9f}
Epoch: 0014 cost= {:.9f}
Epoch: 0015 cost= {:.9f}
Epoch: 0016 cost= {:.9f}
Epoch: 0017 cost= {:.9f}
Epoch: 0018 cost= {:.9f}
Epoch: 0019 cost= {:.9f}
Epoch: 0020 cost= {:.9f}


最后对训练玩的模型进行性能测试，这里使用的是之前定义的成员函数，cal_total_cost对测试集X_test进行测试，评价指标依然是平方误差，如果使用示例中的参数，损失值约为60万

In [54]:
print("Total cost:"+str(autoencoder.calc_total_cost()))

Total cost:648704.75


# 总结
-  去噪自编码器的实现与单隐层神经网络差不多，只是在数据输入时加入了标准化和高斯噪声

- 自编码器是无监督学习的一种方法，它与其他无监督学习的区别是：它不是聚类分析，而是提取最常用最频繁的高阶特征，然后重构数据

